# London Bike-Sharing Datenanalyse

Dieses Notebook bereitet die London Bike-Sharing Daten für die Analyse in Tableau vor. 
Es umfasst folgende Schritte:
1. Daten von Kaggle laden
2. Grundlegende Datenbereinigung
3. Feature Engineering
4. Export für Tableau

## Datenbeschreibung
- timestamp: Zeitstempel der Messung
- cnt: Anzahl der Fahrräder
- t1: Reale Temperatur in °C
- t2: Gefühlte Temperatur in °C
- hum: Luftfeuchtigkeit in %
- wind_speed: Windgeschwindigkeit in km/h
- weather_code: Wetterbedingungen (kodiert)
- is_holiday: Feiertag (0/1)
- is_weekend: Wochenende (0/1)
- season: Jahreszeit (0-3)

In [1]:
# Benötigte Bibliotheken importieren
import pandas as pd          # Für Datenverarbeitung
import zipfile              # Zum Entpacken der Kaggle-Daten
import kaggle               # Für Kaggle API
import openpyxl            # Für Excel-Export

print("Python-Pakete erfolgreich geladen")

Python-Pakete erfolgreich geladen


## 1. Daten laden
Zunächst laden wir die Daten von Kaggle herunter und extrahieren sie.

In [2]:
# Kaggle-Dataset herunterladen
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

# Daten aus ZIP-Datei extrahieren
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

# CSV in DataFrame laden
BIKES = pd.read_csv('london_merged.csv')
print("Daten erfolgreich geladen")

Dataset URL: https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset
License(s): other
london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Daten erfolgreich geladen


## 2. Datenexploration
Überprüfen der Struktur und Qualität der Daten

In [3]:
# Überblick über die Datenstruktur
print("Datenstruktur:")
print("-" * 50)
BIKES.info()

print("\nFehlende Werte:")
print("-" * 50)
print(BIKES.isnull().sum())

Datenstruktur:
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB

Fehlende Werte:
--------------------------------------------------
timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0

In [4]:
# Überprüfung der Datenform
print("Datensatzgröße:", BIKES.shape)

Datensatzgröße: (17414, 10)


## 3. Feature Engineering
Aufbereitung der Daten für die Analyse durch:
1. Vereinheitlichung der Spaltennamen
2. Umwandlung der Wettercodes
3. Zuordnung der Jahreszeiten
4. Normalisierung der Luftfeuchtigkeit

In [5]:
# Spaltennamen für bessere Lesbarkeit anpassen
column_mapping = {
    'timestamp': 'time',
    'cnt': 'counts',
    't1': 'temp_real_c',
    't2': 'temp_feels_like_c',
    'hum': 'humidity_percent',
    'wind_speed': 'wind_speed_kph',
    'weather_code': 'weather',
    'is_holiday': 'is_holiday',
    'is_weekend': 'is_weekend',
    'season': 'season'
}

In [6]:
# Weather Codes in lesbare Bezeichnungen umwandeln
weather_mapping = {
    '1.0': 'Clear',              # Klarer Himmel
    '2.0': 'Scattered Clouds',   # Vereinzelte Wolken
    '3.0': 'Broken Clouds',      # Aufgelockerte Bewölkung
    '4.0': 'Cloudy',            # Bewölkt
    '7.0': 'Rain',              # Regen
    '10.0': 'Rain with Thunderstorm',  # Gewitter
    '26.0': 'Snowfall'          # Schneefall
}

# Jahreszeiten zuordnen
season_mapping = {
    '0.0': 'spring',  # Frühling
    '1.0': 'summer',  # Sommer
    '2.0': 'autumn',  # Herbst
    '3.0': 'winter'   # Winter
}

In [7]:
# Datentransformation durchführen
BIKES.rename(column_mapping, axis=1, inplace=True)
BIKES['weather'] = BIKES['weather'].astype('str').map(weather_mapping)
BIKES['season'] = BIKES['season'].astype('str').map(season_mapping)
BIKES['humidity_percent'] = BIKES['humidity_percent'] / 100  # Normalisierung

## 4. Qualitätsprüfung
Überprüfung der transformierten Daten

In [8]:
# Überprüfen der ersten Zeilen
print("Beispieldaten:")
print("-" * 50)
BIKES.head()

Beispieldaten:
--------------------------------------------------


,time,counts,temp_real_c,temp_feels_like_c,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken Clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


## 5. Daten exportieren
Export der aufbereiteten Daten für Tableau

In [9]:
# Daten als Excel speichern
BIKES.to_excel("london-bikes-final-data.xlsx", sheet_name="data")
print("Daten erfolgreich für Tableau exportiert")

Daten erfolgreich für Tableau exportiert


## Nächste Schritte
Die aufbereiteten Daten können nun in Tableau verwendet werden für:
- Analyse der Nutzungsmuster
- Wettereinfluss-Visualisierung
- Saisonale Trends
- Tages- und Wochenzeitanalysen